---
layout: post
title:  Scala 2 Collections explained
date:   2025-10-27
categories: [Scala]
mermaid: true
maths: true
typora-root-url: /Users/ojitha/GitHub/ojitha.github.io
typora-copy-images-to: ../../blog/assets/images/${filename}
---

<style>
/* Styles for the two-column layout */
.image-text-container {
    display: flex; /* Enables flexbox */
    flex-wrap: wrap; /* Allows columns to stack on small screens */
    gap: 20px; /* Space between the image and text */
    align-items: center; /* Vertically centers content in columns */
    margin-bottom: 20px; /* Space below this section */
}

.image-column {
    flex: 1; /* Allows this column to grow */
    min-width: 250px; /* Minimum width for the image column before stacking */
    max-width: 40%; /* Maximum width for the image column to not take up too much space initially */
    box-sizing: border-box; /* Include padding/border in element's total width/height */
}

.text-column {
    flex: 2; /* Allows this column to grow more (e.g., twice as much as image-column) */
    min-width: 300px; /* Minimum width for the text column before stacking */
    box-sizing: border-box;
}

</style>

<div class="image-text-container">
    <div class="image-column">
        <img src="https://raw.githubusercontent.com/ojitha/blog/master/assets/images/2025-10-26-Scala2-Functors/scala-functors-illustration.svg" alt="Scala Functors" width="150" height="150">
    </div>
    <div class="text-column">
<p>TBC</p>
    </div>
</div>

<!--more-->

------

* TOC
{:toc}
------

## Introduction
Scala's collection library is one of its most powerful features, providing a rich set of data structures and operations for working with sequences, sets, and maps. The collections are designed to be **easy to use**, **concise**, **safe**, **fast**, and **universal**[^1].

## Collection Hierarchy

The Scala collections are organized into three main packages[^2]:

| Package | Description | Mutability |
|---------|-------------|------------|
| `scala.collection` | Base traits and abstract collections | May be immutable or mutable |
| `scala.collection.immutable` | Immutable collections (default) | Never change after creation |
| `scala.collection.mutable` | Mutable collections | Can be modified in place |

```mermaid
---
config:
  look: neo
  theme: default
---
graph TB
    Iterable[Iterable]
    Iterable --> Seq[Seq]
    Iterable --> Set[Set]
    Iterable --> Map[Map]
    
    Seq --> IndexedSeq[IndexedSeq]
    Seq --> LinearSeq[LinearSeq]
    
    IndexedSeq --> Vector
    IndexedSeq --> ArraySeq
    IndexedSeq --> Range
    IndexedSeq --> ArrayBuffer["ArrayBuffer (mutable)"]
    
    LinearSeq --> List
    LinearSeq --> LazyList
    LinearSeq --> Queue["Queue (immutable)"]
    
    Set --> SortedSet
    Set --> HashSet["HashSet"]
    Set --> BitSet
    SortedSet --> TreeSet
    
    Map --> SortedMap
    Map --> HashMap
    Map --> VectorMap
    SortedMap --> TreeMap
    
    style Iterable fill:#e1f5ff
    style Seq fill:#fff4e1
    style Set fill:#ffe1f5
    style Map fill:#e1ffe1
```

### The Iterable Trait

All Scala collections inherit from `Iterable[A]`, which defines an **iterator** that lets you loop through collection elements one at a time[^3]. The iterator can traverse the collection only once, as each element is consumed during iteration.

> **Important**: The `Iterable` trait provides the foundation for all collection operations through its iterator.

## Immutable vs. Mutable Collections

### Immutable Collections (Default)

Immutable collections **never change after creation**[^4]. When you "modify" an immutable collection, you create a new collection with the changes.

Immutable collections are the default:

In [3]:
val set = Set(1,2,3)
val list = List(1,2,3)
val map = Map(1 -> 'a', 2->'b')

set: Set[Int] = Set(1, 2, 3)
list: List[Int] = List(1, 2, 3)
map: Map[Int, Char] = Map(1 -> 'a')

Adding to immutable collections creates new collections:

In [5]:
val set2 = set + 4
val list2 = list :+ 4

set2: Set[Int] = Set(1, 2, 3, 4)
list2: List[Int] = List(1, 2, 3, 4)

Source📝[^5]: Demonstrates how immutable collections are the default in Scala and how they handle additions.

**Logic**: 
- First three lines create immutable collections without any import statements
- The `+` operator on Set and <b>`:+`</b>{:gtxt} operator on List create new collections rather than modifying the originals
- `set` and `list` remain unchanged after operations; only new bindings `set2` and `list2` contain the updated values

>Immutability ensures **referential transparency** - the same expression always evaluates to the same value, making code easier to reason about and thread-safe by default.
{:.green}

### Mutable Collections

Mutable collections can be modified in place using operations that have **side effects**[^6].

Must import or use full path for mutable collections:

In [7]:
import scala.collection.mutable

import scala.collection.mutable

In [13]:
val mutableSet = mutable.Set(1,2,3)

mutableSet: mutable.Set[Int] = HashSet(1, 2, 3)

In [9]:
val mutableList = mutable.ArrayBuffer(1,2,3)

mutableList: mutable.ArrayBuffer[Int] = ArrayBuffer(1, 2, 3)

In [11]:
val mutableMap = mutable.Map(1 -> 'a', 2 -> 'b')

mutableMap: mutable.Map[Int, Char] = HashMap(1 -> 'a', 2 -> 'b')

Modifying mutable collections in place:

In [15]:
mutableSet += 4
mutableList += 4

res14_0: mutable.Set[Int] = HashSet(1, 2, 3, 4)
res14_1: mutable.ArrayBuffer[Int] = ArrayBuffer(1, 2, 3, 4)

Source📝[^7]: Shows how mutable collections differ from immutable ones in both declaration and usage.

> Mutable collections use **side effects**{:rtxt} - they change state rather than creating new values. This can be more efficient for frequent updates but sacrifices thread-safety and referential transparency.
{:.yellow}

### Why Immutability Matters

**Benefits of Immutability**:
- **Thread-safe**: Multiple threads can safely access immutable collections
- **Easier to reason about**: No hidden state changes
- **Referential transparency**: Same inputs always produce same outputs
- **Structural sharing**: Efficient memory use through shared structure

## Main Collection Types

### 1. Sequences (Seq)

Sequences are ordered collections that support indexed access. They branch into two main categories:

#### Linear Sequences (LinearSeq)

Optimised for **head/tail** operations. Elements are accessed sequentially[^8].

List - singly linked list:

In [16]:
val list = List(1, 2, 3, 4, 5)

list: List[Int] = List(1, 2, 3, 4, 5)

Head and tail operations are O(1)

In [17]:
list.head
list.tail

res16_0: Int = 1
res16_1: List[Int] = List(2, 3, 4, 5)

## References

[^1]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Introduction", p. 573

[^2]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 318-319

[^3]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 320

[^4]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Collections Hierarchy", p. 574

[^5]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 323

[^6]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 322

[^7]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 322

[^8]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sequences", p. 321

[^9]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "List Operations"

[^10]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sequences", p. 321-322

[^11]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Understanding the Collections Hierarchy", p. 320

[^12]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Sets", p. 323

[^13]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Creating a Set", p. 453-454

[^14]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Sorted Sets"

[^15]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Maps", p. 322

[^16]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Creating and Using Maps", p. 424

[^17]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Traversing a Map", p. 435-436

[^18]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Transformer Methods", p. 371

[^19]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "map Method", p. 387-389

[^20]: *Programming in Scala, Fourth Edition*, Ch. 24: "Collections in Depth" → "Template Traits", p. 588

[^21]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "filter Method", p. 392-394

[^22]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "flatMap Method"

[^23]: *Functional Programming in Scala*, Ch. 10: "Monoids" → "Folding", p. 180

[^24]: *Functional Programming in Scala*, Ch. 10: "Monoids" → "Associativity and parallelism", p. 179-181

[^25]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "reduce Method", p. 400-405

[^26]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Collection Methods"

[^27]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Taking and Dropping"

[^28]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Grouping Methods", p. 397-399

[^29]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Sorting", p. 410-415

[^30]: *Programming in Scala, Fourth Edition*, Ch. 23: "For Expressions Revisited" → "Translation", p. 557-561

[^31]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "for/yield with Collections", p. 387-390

[^32]: *Programming in Scala, Fourth Edition*, Ch. 23: "For Expressions Revisited" → "Monads", p. 566-567

[^33]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Views", p. 335-337

[^34]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Views", p. 336

[^35]: *Scala Cookbook, Second Edition*, Ch. 11: "Collections: Introduction" → "Performance", p. 330-333

[^36]: *Scala Cookbook, Second Edition*, Ch. 23: "Types" → "Variance", p. 670-674

[^37]: *Scala Cookbook, Second Edition*, Ch. 23: "Types" → "Covariance", p. 671-672

[^38]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Contravariance", p. 425-426

[^39]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Function Variance", p. 426-427

[^40]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Liskov Substitution Principle", p. 425

[^41]: *Programming in Scala, Fourth Edition*, Ch. 19: "Type Parameterization" → "Arrays and Variance", p. 422-423

[^42]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Iterators", p. 381-384

[^43]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Iterators", p. 381-382

[^44]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Ranges", p. 449-453

[^45]: *Scala Cookbook, Second Edition*, Ch. 15: "Collections: Tuple, Range, Set, Stack, and Queue" → "Ranges", p. 452-453

[^46]: *Advanced Scala with Cats*, Ch. 3: "Functors" → "Maybe Monad"

[^47]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "Option Integration"

[^48]: Philip Wadler, "Comprehending Monads", p. 22, *WadlerMonads.pdf*

[^49]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → Examples

[^50]: *Scala Cookbook, Second Edition*, Ch. 13: "Collections: Common Sequence Methods" → "flatMap examples"

[^51]: *Scala Cookbook, Second Edition*, Ch. 14: "Collections: Using Maps" → "Immutable Map Updates"

{:gtxt: .message color="green"}
{:ytxt: .message color="yellow"}
{:rtxt: .message color="red"}